[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meta-llama/llama-stack/blob/main/docs/getting_started.ipynb)

# Llama Stack - Building AI Applications

<img src="https://llama-stack.readthedocs.io/en/latest/_images/llama-stack.png" alt="drawing" width="500"/>

[Llama Stack](https://github.com/meta-llama/llama-stack) defines and standardizes the set of core building blocks needed to bring generative AI applications to market. These building blocks are presented in the form of interoperable APIs with a broad set of Service Providers providing their implementations.

Read more about the project here: https://llama-stack.readthedocs.io/en/latest/index.html

In this guide, we will showcase how you can build LLM-powered agentic applications using Llama Stack.

**💡 Quick Start Option:** If you want a simpler and faster way to test out Llama Stack, check out the [quick_start.ipynb](quick_start.ipynb) notebook instead. It provides a streamlined experience for getting up and running in just a few steps.


## 1. Getting started with Llama Stack

### 1.1. Create TogetherAI account


In order to run inference for the llama models, you will need to use an inference provider. Llama stack supports a number of inference [providers](https://github.com/meta-llama/llama-stack/tree/main/llama_stack/providers/remote/inference).


In this showcase, we will use [together.ai](https://www.together.ai/) as the inference provider. So, you would first get an API key from Together if you dont have one already.

Steps [here](https://docs.google.com/document/d/1Vg998IjRW_uujAPnHdQ9jQWvtmkZFt74FldW2MblxPY/edit?usp=sharing).

You can also use Fireworks.ai or even Ollama if you would like to.



> **Note:**  Set the API Key in the Secrets of this notebook



### 1.2. Setup and Running a Llama Stack server

Llama Stack is architected as a collection of APIs that provide developers with the building blocks to build AI applications.

Llama stack is typically available as a server with an endpoint that you can make calls to. Partners like Together and Fireworks offer their own Llama Stack compatible endpoints.

In this showcase, we will start a Llama Stack server that is running locally.


In [1]:
#Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

#Start Ollama server with llama3 model
!nohup ollama serve > ollama_server.log 2>&1 &
!ollama pull llama-guard3:1b
!ollama pull llama3.2:3b

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.




In [2]:
!curl 127.0.0.1:11434/v1/models


{"object":"list","data":[{"id":"llama3.2:3b","object":"model","created":1758304995,"owned_by":"library"},{"id":"llama-guard3:1b","object":"model","created":1758304963,"owned_by":"library"}]}


In [ ]:
# use this helper if needed to kill the server
!rm -rf ~/.llama/distributions/*
import os
def kill_llama_stack_server():
    # Kill any existing llama stack server processes
    os.system("ps aux | grep -v grep | grep llama_stack.core.server.server | awk '{print $2}' | xargs kill -9")
kill_llama_stack_server()

In [1]:
# Install UV if not available
!curl -LsSf https://astral.sh/uv/install.sh | sh
# Complete setup for Google Colab with custom directories
import os
!rm -rf /content/llama-project
# Set environment variables
os.environ['UV_CACHE_DIR'] = '/content/uv-cache'
os.environ['UV_PROJECT_DIR'] = '/content/llama-project'
os.environ['OLLAMA_URL'] = 'http://localhost:11434'
# Create directories
!mkdir -p /content/uv-cache
!mkdir -p /content/llama-project
!cd /content/llama-project && uv venv venv
!source /content/llama-project/venv/bin/activate && uv run --with llama-stack==0.2.22 llama stack build --distro starter-gpu --image-type venv
!nohup python -m llama_stack.core.server.server /root/.llama/distributions/starter-gpu/starter-gpu-run.yaml --port 8321 > llama_stack_server.log &
def wait_for_server_to_start():
    import requests
    from requests.exceptions import ConnectionError
    import time

    url = "http://0.0.0.0:8321/v1/health"
    max_retries = 30
    retry_interval = 1

    print("Waiting for server to start", end="")
    for _ in range(max_retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                print("\nServer is ready!")
                return True
        except ConnectionError:
            print(".", end="", flush=True)
            time.sleep(retry_interval)

    print("\nServer failed to start after", max_retries * retry_interval, "seconds")
    return False
assert wait_for_server_to_start()
print("llama stack server hosted on localhost:8321")

downloading uv 0.8.19 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Using CPython 3.12.11 interpreter at: /usr/bin/python3
Creating virtual environment at: venv
Activate with: source venv/bin/activate
Installed 84 packages in 334ms
WARNING  2025-09-19 23:29:10,361 llama_stack.core.distribution:149 core: Failed to import module prompts: No module named                             
         'llama_stack.providers.registry.prompts'                                                                                                     
Environment '/content/uv-cache/builds-v0/.tmpJfVJ5w' already exists, re-using it.
Installing dependencies in system Python environment
Using Python 3.12.11 environment at: /usr
Resolved 84 packages in 1.30s
Prepared 12 packages in 318ms
Uninstalled 1 package in 9ms
Installed 12 packages in 38ms
 + aiosqlite==0.21.0
 + asyncpg==0.30.0
 + ecdsa==0.19.1
 + fire==0.7.1
 + llama-stack==0.2.22
 + llama-

### 1.4. Install and Configure the Client

Now that we have our Llama Stack server running locally, we need to install the client package to interact with it. The `llama-stack-client` provides a simple Python interface to access all the functionality of Llama Stack, including:

- Chat Completions ( text and multimodal )
- Safety Shields
- Agent capabilities with tools like web search, RAG with Telemetry
- Evaluation and scoring frameworks

The client handles all the API communication with our local server, making it easy to integrate Llama Stack's capabilities into your applications.

In the next cells, we'll:

1. Install the client package
2. Set up API keys for external services (Together AI and Tavily Search)
3. Initialize the client to connect to our local server


In [3]:
import os
import getpass
try:
    from google.colab import userdata
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
    os.environ['TAVILY_SEARCH_API_KEY'] = userdata.get('TAVILY_SEARCH_API_KEY')
except ImportError:
    print("Not in Google Colab environment")

for key in ['GROQ_API_KEY', 'TAVILY_SEARCH_API_KEY']:
    try:
        api_key = os.environ[key]
        if not api_key:
            raise ValueError(f"{key} environment variable is empty")
    except KeyError:
        api_key = getpass.getpass(f"{key} environment variable is not set. Please enter your API key: ")
        os.environ[key] = api_key

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url="http://0.0.0.0:8321",
    provider_data = {
        "tavily_search_api_key": os.environ['TAVILY_SEARCH_API_KEY'],
        "groq_api_key": os.environ['GROQ_API_KEY']
    }
)

Now that we have completed the setup and configuration, let's start exploring the capabilities of Llama Stack! We'll begin by checking what models and safety shields are available, and then move on to running some example chat completions.



### 1.5. Check available models and shields

All the models available in the provider are now programmatically accessible via the client.

In [4]:
from rich.pretty import pprint

print("Available models:")
for m in client.models.list():
    print(f"- {m.identifier}")



Available models:
- fireworks/accounts/fireworks/models/llama-v3p1-8b-instruct
- fireworks/accounts/fireworks/models/llama-v3p1-70b-instruct
- fireworks/accounts/fireworks/models/llama-v3p1-405b-instruct
- fireworks/accounts/fireworks/models/llama-v3p2-3b-instruct
- fireworks/accounts/fireworks/models/llama-v3p2-11b-vision-instruct
- fireworks/accounts/fireworks/models/llama-v3p2-90b-vision-instruct
- fireworks/accounts/fireworks/models/llama-v3p3-70b-instruct
- fireworks/accounts/fireworks/models/llama4-scout-instruct-basic
- fireworks/accounts/fireworks/models/llama4-maverick-instruct-basic
- fireworks/nomic-ai/nomic-embed-text-v1.5
- fireworks/accounts/fireworks/models/llama-guard-3-8b
- fireworks/accounts/fireworks/models/llama-guard-3-11b-vision
- bedrock/meta.llama3-1-8b-instruct-v1:0
- bedrock/meta.llama3-1-70b-instruct-v1:0
- bedrock/meta.llama3-1-405b-instruct-v1:0
- openai/gpt-3.5-turbo-0125
- openai/gpt-3.5-turbo
- openai/gpt-3.5-turbo-instruct
- openai/gpt-4
- openai/gpt-4-

### 1.6. Run a simple chat completion with one of the models

We will test the client by doing a simple chat completion.

In [5]:
#model_id = "ollama/llama3.2:3b"
model_id = "groq/meta-llama/llama-4-maverick-17b-128e-instruct"
response = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "You are a friendly assistant."},
        {"role": "user", "content": "Write a two-sentence poem about llama."},
    ],
    stream=False
)

print(response.choices[0].message.content)


Here's a two-sentence poem about a llama:

With gentle eyes and soft, fuzzy hair, the llama roams with gentle, peaceful air. In the Andes, it climbs with steady pace, a serene and majestic animal in its sacred space.


### 1.7. Have a conversation

Maintaining a conversation history allows the model to retain context from previous interactions. Use a list to accumulate messages, enabling continuity throughout the chat session.

In [6]:
from termcolor import cprint

questions = [
    "Who was the most famous PM of England during world war 2 ?",
    "What was his most famous quote ?"
]


def chat_loop():
    conversation_history = []
    while len(questions) > 0:
        user_input = questions.pop(0)
        if user_input.lower() in ["exit", "quit", "bye"]:
            cprint("Ending conversation. Goodbye!", "yellow")
            break

        user_message = {"role": "user", "content": user_input}
        conversation_history.append(user_message)

        response = client.chat.completions.create(
            messages=conversation_history,
            model=model_id,
        )
        cprint(f"> Response: {response.choices[0].message.content}", "cyan")

        assistant_message = {
            "role": "assistant",  # was user
            "content": response.choices[0].message.content,
            "finish_reason": response.choices[0].finish_reason,
        }
        conversation_history.append(assistant_message)


chat_loop()


> Response: You're likely thinking of Winston Churchill!

Winston Churchill was indeed the most famous Prime Minister of the United Kingdom during World War II. He served as the Prime Minister from May 10, 1940, to July 26, 1945, and again from 1951 to 1955. Churchill played a crucial role in leading Britain through the war, rallying the British people with his inspiring speeches, and forming alliances with other countries to defeat the Axis powers.

Churchill's leadership, oratory skills, and unwavering resolve made him a iconic figure of the war era, and he remains one of the most revered and celebrated leaders in British history.

Is there anything else you'd like to know about Churchill or his role during World War II?
> Response: One of the most famous quotes attributed to Winston Churchill is:

"We shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the streets, we shall fight in the hills; we shall never surrender."

This quote 

Here is an example for you to try a conversation yourself.
Remember to type `quit` or `exit` after you are done chatting.

In [ ]:
# NBVAL_SKIP
from termcolor import cprint

def chat_loop():
    conversation_history = []
    while True:
        user_input = input("User> ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            cprint("Ending conversation. Goodbye!", "yellow")
            break

        user_message = {"role": "user", "content": user_input}
        conversation_history.append(user_message)

        response = client.chat.completions.create(
            messages=conversation_history,
            model=model_id,
        )
        cprint(f"> Response: {response.choices[0].message.content}", "cyan")

        assistant_message = {
            "role": "assistant",  # was user
            "content": response.choices[0].message.content,
            "finish_reason": response.choices[0].finish_reason,
        }
        conversation_history.append(assistant_message)


chat_loop()


User> who are you?
> Response: I'm an AI assistant designed by Meta. I'm here to answer your questions, share interesting ideas and maybe even surprise you with a fresh perspective. What's on your mind?
User> how can you help me?
> Response: I can help you with a wide range of things, such as answering questions, providing information, generating text or images, summarizing content, or just having a chat. I can also help with creative tasks like brainstorming or coming up with ideas. What do you need help with today?
User> bye
Ending conversation. Goodbye!


### 1.9. Streaming output

You can pass `stream=True` to stream responses from the model. You can then loop through the responses.

In [ ]:
from llama_stack_client import InferenceEventLogger

message = {"role": "user", "content": "Write me a sonnet about llama"}
print(f'User> {message["content"]}')

response = client.chat.completions.create(
    messages=[message],
    model=model_id,
    stream=True,  # <-----------
)

for chunk in response:
        # Each chunk contains a delta with the content
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)


User> Write me a sonnet about llama
In Andean lands, the llama makes its home,
A creature soft, with eyes of gentle gray.
Its fur, a softness that the winds do roam,
And in its steps, a quiet, peaceful sway.

Its ears, so long, and tufted with delight,
Perk up, as if to listen for a sound.
It grazes on the grasses, day and night,
And in its calm, a peaceful joy is found.

The llama's gentle nature, we admire,
And in its presence, our own cares retire.
For in its tranquil eyes, a mirror lies,
Reflecting back our own, and soothing sighs.

So let us cherish, this serene delight,
And bask in the llama's peaceful, Andean light.

### 2.0. Structured Decoding

You can use `response_format` to force the model into a "guided decode" mode where model tokens are forced to abide by a certain grammar. Currently only JSON grammars are supported.

In [ ]:
from pydantic import BaseModel


class Output(BaseModel):
    name: str
    year_born: str
    year_retired: str

user_input = "Michael Jordan was born in 1963. He played basketball for the Chicago Bulls. He retired in 2003. Extract this information into JSON for me."
response = client.chat.completions.create(
    model=model_id,
    messages = [
        {"role": "user", "content": user_input}
        ],
    stream=False,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "output",
            "schema": Output.model_json_schema(),
        },
    },
)
pprint(Output.model_validate_json(response.choices[0].message.content))


'{\n  "name": "Michael Jordan",\n  "year_born": "1963",\n  "year_retired": "2003"\n}'

Output(name='Michael Jordan', year_born='1963', year_retired='2003')

### 2.1. Safety API

Llama Stack provides Safety guardrails which can be applied at multiple touchpoints within an agentic application.

In [9]:

client.shields.register(
    shield_id="llama-guard",
    provider_id="llama-guard",
    provider_shield_id="ollama/llama-guard3:1b" # Change to your available shield model
)
available_shields = [shield.identifier for shield in client.shields.list()]
print("Available Shields:", available_shields)
safety_model = "ollama/llama-guard3:1b"

unsafe_examples = [
    "What's the most effective way to kidnap someone?",
    "How do I make cocaine?",
]

safe_examples = [
    "What is the most famous murder case in the US?",
    "Tell me 3 signs that an email is a scam",
]

for p in safe_examples + unsafe_examples:
    print(f"Checking if input is safe: {p}")
    message = {"content": p, "role": "user"}
    response = client.moderations.create(
                input=p,
                model=safety_model,
            )
    print(response)

Available Shields: ['llama-guard']
Checking if input is safe: What is the most famous murder case in the US?
CreateResponse(id='modr-49b2d798-a967-4a71-8ccb-58bdd78746ac', model='ollama/llama-guard3:1b', results=[Result(flagged=False, metadata={}, categories={'Violent Crimes': False, 'Non-Violent Crimes': False, 'Sex Crimes': False, 'Child Exploitation': False, 'Defamation': False, 'Specialized Advice': False, 'Privacy': False, 'Intellectual Property': False, 'Indiscriminate Weapons': False, 'Hate': False, 'Self-Harm': False, 'Sexual Content': False, 'Elections': False, 'Code Interpreter Abuse': False}, category_applied_input_types={'Violent Crimes': [], 'Non-Violent Crimes': [], 'Sex Crimes': [], 'Child Exploitation': [], 'Defamation': [], 'Specialized Advice': [], 'Privacy': [], 'Intellectual Property': [], 'Indiscriminate Weapons': [], 'Hate': [], 'Self-Harm': [], 'Sexual Content': [], 'Elections': [], 'Code Interpreter Abuse': []}, category_scores={'Violent Crimes': 1.0, 'Non-Viole

In [ ]:
!llama-stack-client shields list

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/shields "HTTP/1.1 200 OK"


## 2. Llama Stack Agents

Llama Stack provides all the building blocks needed to create sophisticated AI applications. This guide will walk you through how to use these components effectively.




<img src="https://github.com/meta-llama/llama-stack/blob/main/docs/resources/agentic-system.png?raw=true" alt="drawing" width="800"/>


Agents are characterized by having access to

1. Memory - for RAG
2. Tool calling - ability to call tools like search and code execution
3. Tool call + Inference loop - the LLM used in the agent is able to perform multiple iterations of call
4. Shields - for safety calls that are executed everytime the agent interacts with external systems, including user prompts

### 2.1. List available tool groups on the provider

In [ ]:
from rich.pretty import pprint
for toolgroup in client.toolgroups.list():
    pprint(toolgroup)

ToolGroup(
│   identifier='builtin::rag',
│   provider_id='rag-runtime',
│   type='tool_group',
│   args=None,
│   mcp_endpoint=None,
│   provider_resource_id='builtin::rag'
)

ToolGroup(
│   identifier='builtin::websearch',
│   provider_id='tavily-search',
│   type='tool_group',
│   args=None,
│   mcp_endpoint=None,
│   provider_resource_id='builtin::websearch'
)

### 2.2. Search agent

In this example, we will show how the model can invoke search to be able to answer questions. We will first have to set the API key of the search tool.

Let's make sure we set up a web search tool for the model to call in its agentic loop. In this tutorial, we will use [Tavily](https://tavily.com) as our search provider. Note that the "type" of the tool is still "brave_search" since Llama models have been trained with brave search as a builtin tool. Tavily is just being used in lieu of Brave search.

See steps [here](https://docs.google.com/document/d/1Vg998IjRW_uujAPnHdQ9jQWvtmkZFt74FldW2MblxPY/edit?tab=t.0#heading=h.xx02wojfl2f9).

In [ ]:
from llama_stack_client import Agent, AgentEventLogger
from termcolor import cprint

web_search_response = client.responses.create(
    model=model_id,
    input="Which teams played in the NBA western conference finals of 2024",
    tools=[
        {
            "type": "web_search",
        },
    ],  # Web search for current information
)
print(f"Web search results: {web_search_response.output[-1].content[0].text}")

Web search results: The teams that played in the 2024 NBA Western Conference Finals were the Dallas Mavericks and the Minnesota Timberwolves. The Mavericks won the series 4-1.


### 2.3. RAG Agent

In this example, we will index some documentation and ask questions about that documentation.

The tool we use is the memory tool. Given a list of memory banks,the tools can help the agent query and retireve relevent chunks. In this example, we first create a memory bank and add some documents to it. Then configure the agent to use the memory tool. The difference here from the websearch example is that we pass along the memory bank as an argument to the tool. A toolgroup can be provided to the agent as just a plain name, or as a dict with both name and arguments needed for the toolgroup. These args get injected by the agent for every tool call that happens for the corresponding toolgroup.

In [ ]:
from io import BytesIO


#delete any existing vector store
vector_stores_to_delete = [v.id for v in client.vector_stores.list()]
for del_vs_id in vector_stores_to_delete:
    client.vector_stores.delete(vector_store_id=del_vs_id)
print('Deleted all exisitng vector store')

docs = [
    ("Acme ships globally in 3-5 business days.", {"title": "Shipping Policy"}),
    ("Returns are accepted within 30 days of purchase.", {"title": "Returns Policy"}),
    ("Support is available 24/7 via chat and email.", {"title": "Support"}),
]
query = "How long does shipping take?"
file_ids = []
for content, metadata in docs:
  with BytesIO(content.encode()) as file_buffer:
      file_buffer.name = f"{metadata['title'].replace(' ', '_').lower()}.txt"
      create_file_response = client.files.create(file=file_buffer, purpose="assistants")
      print(create_file_response)
      file_ids.append(create_file_response.id)

# Create vector store with files
vector_store = client.vector_stores.create(
  name="acme_docs",
  file_ids=file_ids,
  embedding_model="sentence-transformers/all-MiniLM-L6-v2",
  embedding_dimension=384,
  provider_id="faiss"
)
print("Listing available vector stores:")
vector_stores = client.vector_stores.list()
for vs in vector_stores:
    print(f"- {vs.name} (ID: {vs.id})")
    files_in_store = client.vector_stores.files.list(vector_store_id=vs.id)
    if files_in_store:
        print(f"  - Files in vector store '{vs.name}' (ID: {vs.id}):")
        for file in files_in_store:
            print(f"- {file.id}")
print("Searching Vector_store with query")
file_search_response = client.responses.create(
    model=model_id,
    input=query,
    tools=[
        {  # Using Responses API built-in tools
            "type": "file_search",
            "vector_store_ids": [vector_store.id],  # Vector store containing uploaded files
        },
    ],
)
print(file_search_response)
print(f"File search results: {file_search_response.output[-1].content[0].text}")


Deleted all exisitng vector store
File(id='file-354f3e6b09974322b5ad0007d5ece533', bytes=41, created_at=1758228715, expires_at=1789764715, filename='shipping_policy.txt', object='file', purpose='assistants')
File(id='file-94933acc81c043c9984d912736235294', bytes=48, created_at=1758228715, expires_at=1789764715, filename='returns_policy.txt', object='file', purpose='assistants')
File(id='file-540a598305114c1b90f68142cae56dc8', bytes=45, created_at=1758228715, expires_at=1789764715, filename='support.txt', object='file', purpose='assistants')
Listing available vector stores:
- acme_docs (ID: vs_4fba2b6a-0123-40c2-9dcf-61b6c50ec8c9)
  - Files in vector store 'acme_docs' (ID: vs_4fba2b6a-0123-40c2-9dcf-61b6c50ec8c9):
- file-354f3e6b09974322b5ad0007d5ece533
- file-94933acc81c043c9984d912736235294
- file-540a598305114c1b90f68142cae56dc8
Searching Vector_store with query
ResponseObject(id='resp-543f47fd-5bda-459d-8d61-39383a34bcf0', created_at=1758228715, model='groq/llama-3.1-8b-instant', ob

### 2.4. Using Model Context Protocol

In this example, we will show how tools hosted in an MCP server can be configured to be used by the model.

In the following steps, we will use the [filesystem tool](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem) to explore the files and folders available in the /content directory

Use xterm module to start a shell to run the MCP server using the `supergateway` tool which can start an MCP tool and serve it over HTTP.

### 2.4. Using Model Context Protocol


This section demonstrates how to use the Model Context Protocol (MCP) with Llama Stack to interact with external tools hosted on an MCP server.


- This example demonstrates how to use the Llama Stack client to interact with a remote MCP tool.
- In this specific example, it connects to a remote Cloudflare documentation MCP server (`https://docs.mcp.cloudflare.com/sse`).
- The `client.responses.create` method is used with the `mcp` tool type, specifying the server details and the user input ("what is cloudflare").


**Key Concepts:**

- **Model Context Protocol (MCP):** A protocol that allows language models to interact with external tools and services.
- **MCP Tool:** A specific tool (like filesystem or a dice roller) that adheres to the MCP and can be interacted with by an MCP-enabled agent.
- **`client.responses.create`:** The Llama Stack client method used to create a response from a model, which can include tool calls to MCP tools.

This setup provides a flexible way to extend the capabilities of your Llama Stack agents by integrating with various external services and tools via the Model Context Protocol.

In [7]:
# NBVAL_SKIP
resp = client.responses.create(
    model=model_id,
    tools=[
        {
            "type": "mcp",
            "server_label": "cloudflare_docs",
            "server_description": "A MCP server for cloudflare documentation.",
            "server_url": "https://docs.mcp.cloudflare.com/sse",
            "require_approval": "never",
        },
    ],
    input="what is cloudflare",
)

print(resp.output_text)

Cloudflare is a cloud-based service that provides a range of features to help protect and improve the performance, security, and reliability of websites, applications, and other online services. It is one of the world's largest connectivity cloud networks, powering Internet requests for millions of websites and serving 55 million HTTP requests per second on average.

Some of the key things Cloudflare does include:

1. Content Delivery Network (CDN): caching website content across a network of servers worldwide to reduce load times.
2. DDoS Protection: protecting against Distributed Denial-of-Service attacks by filtering out malicious traffic.
3. Firewall: acting as an additional layer of security, filtering out hacking attempts and malicious traffic.
4. SSL Encryption: providing free SSL encryption to secure sensitive information.
5. Bot Protection: identifying and blocking bots trying to exploit vulnerabilities or scrape content.
6. Analytics: providing insights into website traffic t

## 3. Llama Stack Agent Evaluations


#### 3.1. Online Evaluation Dataset Collection

- Llama Stack allows you to query each steps of the agents execution in your application.
- In this example, we will show how to
    1. build an Agent with Llama Stack
    2. Query the agent's session, turns, and steps
    3. Evaluate the results

##### 3.1.1. Building a Search Agent

First, let's build an agent that have access to a search tool with Llama Stack, and use it to run some user queries.

In [ ]:
from llama_stack_client import Agent, AgentEventLogger

agent = Agent(
    client,
    model="together/meta-llama/Llama-3.3-70B-Instruct-Turbo",
    instructions="You are a helpful assistant. Use web_search tool to answer the questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    "Which teams played in the NBA western conference finals of 2024. Search the web for the answer.",
    "In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.",
    "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
]

session_id = agent.create_session(uuid.uuid4().hex)

for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()


inference> brave_search.call(query="NBA Western Conference Finals 2024 teams")
tool_execution> Tool:brave_search Args:{'query': 'NBA Western Conference Finals 2024 teams'}
tool_execution> Tool:brave_search Response:{"query": "NBA Western Conference Finals 2024 teams", "top_k": [{"url": "https://www.basketball-reference.com/playoffs/NBA_2024.html", "title": "2024 NBA Playoffs Summary", "content": "Western Conference Finals, Dallas Mavericks over Minnesota Timberwolves (4-1), Series Stats \u00b7 Game 1, Wed, May 22, Dallas Mavericks, 108, @ Minnesota", "score": 0.8849276, "raw_content": null}, {"url": "https://www.basketball-reference.com/playoffs/2024-nba-western-conference-finals-mavericks-vs-timberwolves.html", "title": "2024 NBA Western Conference Finals - Mavericks vs. ...", "content": "# 2024 NBA Western Conference Finals Mavericks vs. * 2024 NBA Playoffs + Dallas Mavericks vs. + Dallas Mavericks vs. + Minnesota Timberwolves vs. + Dallas Mavericks vs. + Dallas Mavericks vs. + Dalla

##### 3.1.2 Query Agent Execution Steps

Now, let's look deeper into the agent's execution steps and see if how well our agent performs. As a sanity check, we will first check if all user prompts is followed by a tool call to `brave_search`.

In [ ]:
# query the agents session
from rich.pretty import pprint

session_response = client.agents.session.retrieve(
    session_id=session_id,
    agent_id=agent.agent_id,
)

pprint(session_response.turns)

In [ ]:
num_tool_call = 0
for turn in session_response.turns:
    for step in turn.steps:
        if step.step_type == "tool_execution" and step.tool_calls[0].tool_name == "brave_search":
            num_tool_call += 1

print(f"{num_tool_call}/{len(session_response.turns)} user prompts are followed by a tool call to `brave_search`")

3/3 user prompts are followed by a tool call to `brave_search`


##### 3.1.3 Evaluate Agent Responses

Now, we want to evaluate the agent's responses to the user prompts.

1. First, we will process the agent's execution history into a list of rows that can be used for evaluation.
2. Next, we will label the rows with the expected answer.
3. Finally, we will use the `/scoring` API to score the agent's responses.

In [ ]:
eval_rows = []

expected_answers = [
    "Dallas Mavericks and the Minnesota Timberwolves",
    "Season 4, Episode 12",
    "King Cobra",
]

for i, turn in enumerate(session_response.turns):
    eval_rows.append(
        {
            "input_query": turn.input_messages[0].content,
            "generated_answer": turn.output_message.content,
            "expected_answer": expected_answers[i],
        }
    )

pprint(eval_rows)

scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

[
│   {
│   │   'input_query': 'Which teams played in the NBA western conference finals of 2024. Search the web for the answer.',
│   │   'generated_answer': 'The teams that played in the NBA Western Conference Finals of 2024 were the Dallas Mavericks and the Minnesota Timberwolves.',
│   │   'expected_answer': 'Dallas Mavericks and the Minnesota Timberwolves'
│   },
│   {
│   │   'input_query': 'In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.',
│   │   'generated_answer': 'Bill Cosby first appears in the episode "Trapped in the Closet" (Season 9, Episode 12) of South Park.',
│   │   'expected_answer': 'Season 4, Episode 12'
│   },
│   {
│   │   'input_query': "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
│   │   'generated_answer': 'Andrew Tate\'s kickboxing name is "King Cobra".',
│   │   'expected_answer': 'King Cobra'
│   }
]

ScoringScoreResponse(
│   results={
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 0.6666666666666666, 'num_correct': 2.0, 'num_total': 3}},
│   │   │   score_rows=[{'score': 1.0}, {'score': 0.0}, {'score': 1.0}]
│   │   )
│   }
)

##### 3.1.4 Query Telemetry & Evaluate

Another way to get the agent's execution history is to query the telemetry logs from the `/telemetry` API. The following example shows how to query the telemetry logs and post-process them to prepare data for evaluation.

In [ ]:
# NBVAL_SKIP
print(f"Getting traces for session_id={session_id}")
import json

from rich.pretty import pprint

agent_logs = []

for span in client.telemetry.query_spans(
    attribute_filters=[
        {"key": "session_id", "op": "eq", "value": session_id},
    ],
    attributes_to_return=["input", "output"],
):
    if span.attributes["output"] != "no shields":
        agent_logs.append(span.attributes)

print("Here are examples of traces:")
pprint(agent_logs[:2])


Getting traces for session_id=d73d9aaa-65ac-4255-8153-9f5cbff6e01e
Here are examples of traces:


[
│   {
│   │   'input': '[{"role": "system", "content": "You are a helpful assistant. Use web_search tool to answer the questions."}, {"role": "user", "content": "Which teams played in the NBA western conference finals of 2024. Search the web for the answer.", "context": null}]',
│   │   'output': '{"content": "", "tool_calls": [{"call_id": "5f77ab69-72d9-4d51-b96c-bd4352ced54a", "tool_name": "brave_search", "arguments": {"query": "NBA Western Conference Finals 2024 teams"}, "arguments_json": "{\\"query\\": \\"NBA Western Conference Finals 2024 teams\\"}"}]}'
│   },
│   {
│   │   'input': '{"role":"assistant","content":"","stop_reason":"end_of_turn","tool_calls":[{"call_id":"5f77ab69-72d9-4d51-b96c-bd4352ced54a","tool_name":"brave_search","arguments":{"query":"NBA Western Conference Finals 2024 teams"},"arguments_json":"{\\"query\\": \\"NBA Western Conference Finals 2024 teams\\"}"}]}',
│   │   'output': '{"role":"tool","call_id":"5f77ab69-72d9-4d51-b96c-bd4352ced54a","content":"{\\"query\\": \\"NBA Western Conference Finals 2024 teams\\", \\"top_k\\": [{\\"title\\": \\"2024 NBA Western Conference Finals - Basketball-Reference.com\\", \\"url\\": \\"https://www.basketball-reference.com/playoffs/2024-nba-western-conference-finals-mavericks-vs-timberwolves.html\\", \\"content\\": \\"2024 NBA Playoffs Dallas Mavericks vs. Dallas Mavericks vs. Dallas Mavericks vs. 5 Dallas Mavericks (4-1) vs. 7   Derrick Jones Jr. 2024 NBA Playoffs Dallas Mavericks vs. Dallas Mavericks vs. Dallas Mavericks vs. College Tools: Player Season Finder, Player Game Finder, Team Season Finder, Team Game Finder Players, Teams, Seasons, Leaders, Awards ... Players, Teams, Seasons, Leaders, Awards ... Players, Teams, Seasons, Leaders, Awards, All-Star Games, Executives ... Players, Teams, Seasons, Leaders, Awards ... Subscribe to Stathead Basketball: Get your first month FREE The SPORTS REFERENCE, STATHEAD, IMMACULATE GRID, and IMMACULATE FOOTY trademarks are owned exclusively by Sports Reference LLC. Sports\\\\u00a0Reference\\\\u202f\\\\u00ae Baseball Football (college) Basketball (college) Hockey F\\\\u00fatbol Blog Stathead\\\\u202f\\\\u00ae Immaculate Grid\\\\u202f\\\\u00ae\\", \\"score\\": 0.89030397, \\"raw_content\\": null}, {\\"title\\": \\"NBA Standings - 2024-25 season - ESPN\\", \\"url\\": \\"https://www.espn.com/nba/standings\\", \\"content\\": \\"NBA Standings - 2024-25 season - ESPN Skip to main contentSkip to navigation ESPN NFL NBA NCAAF NHL NCAAM NCAAW Soccer More Sports Watch Fantasy NBA Home Scores Schedule Standings Stats Teams Odds Where To Watch All-Star Game Fantasy More NBA Standings 2024-25 Standings Expanded Vs. Division NBA Cup LeagueConferenceDivision Eastern Conference | | | --- | | 1CLECleveland Cavaliers | | 2BOSBoston Celtics | | 3NYNew York Knicks | | 4INDIndiana Pacers | | 5MILMilwaukee Bucks | | 6DETDetroit Pistons | | 7MIAMiami Heat | | 8ORLOrlando Magic | | 9ATLAtlanta Hawks | | 10CHIChicago Bulls | | PHIPhiladelphia 76ers | | BKNBrooklyn Nets | | TORToronto Raptors | | CHACharlotte Hornets | | WSHWashington Wizards | | W | L | PCT | GB | HOME | AWAY | DIV | CONF | PPG | OPP PPG | DIFF | STRK | L10 | | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | | 42 | 10 | .808 | - | 24-4 | 18-6 | 9-1 | 28-7 | 122.4 | 112.1 | +10.3 | W2 | 6-4 | | 36 | 16 | .692 | 6 | 16-10 | 20-6 | 6-2 | 26-9 | 117.3 | 108.8 | +8.5 | L1 | 7-3 | | 34 | 17 | .667 | 7.5 | 18-9 | 16-8 | 9-1 | 23-10 | 117.9 | 111.4 | +6.5 | W2 | 8-2 | | 29 | 21 | .580 | 12 | 14-7 | 14-13 | 6-4 | 17-15 | 115.7 | 114.9 | +0.8 | W1 | 7-3 | | 27 | 23 | .540 | 14 | 16-8 | 10-15 | 6-5 | 22-16 | 114.2 | 112.6 | +1.6 | L1 | 4-6 | | 26 | 26 | .500 | 16 | 13-13 | 13-13 | 2-9 | 18-20 | 113.0 | 113.8 | -0.8 | W1 | 5-5 | | 25 | 25 | .500 | 16 | 12-10 | 12-15 | 5-3 | 14-15 | 110.5 | 110.6 | -0.1 | L1 | 5-5 | | 25 | 28 | .472 | 17.5 | 15-9 | 10-19 | 5-2 | 20-15 | 103.8 | 105.6 | -1.8 | L1 | 2-8 | | 24 | 28 | .462 | 18 | 12-12 | 12-15 | 4-2 | 17-13 | 116.1 |

- Now, we want to run evaluation to assert that our search agent succesfully calls brave_search from online traces.
- We will first post-process the agent's telemetry logs and run evaluation.

In [ ]:
# NBVAL_SKIP
# post-process telemetry spance and prepare data for eval
# in this case, we want to assert that all user prompts is followed by a tool call
import ast
import json

eval_rows = []

for log in agent_logs:
    input = json.loads(log["input"])
    if isinstance(input, list):
        input = input[-1]
    if input["role"] == "user":
        eval_rows.append(
            {
                "input_query": input["content"],
                "generated_answer":  log["output"],
                # check if generated_answer uses tools brave_search
                "expected_answer": "brave_search",
            },
        )

# pprint(eval_rows)
scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)


ScoringScoreResponse(
│   results={
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 1.0, 'num_correct': 3.0, 'num_total': 3}},
│   │   │   score_rows=[{'score': 1.0}, {'score': 1.0}, {'score': 1.0}]
│   │   )
│   }
)

#### 3.2. Agentic Application Dataset Scoring
- Llama Stack offers a library of scoring functions and the `/scoring` API, allowing you to run evaluations on your pre-annotated AI application datasets.

- In this example, we will work with an example RAG dataset you have built previously, label with an annotation, and use LLM-As-Judge with custom judge prompt for scoring. Please checkout our [Llama Stack Playground](https://llama-stack.readthedocs.io/en/latest/playground/index.html) for an interactive interface to upload datasets and run scorings.

In [ ]:
import rich
from rich.pretty import pprint

# could even use larger models like 405B
judge_model_id = "together/meta-llama/Llama-3.3-70B-Instruct-Turbo"

JUDGE_PROMPT = """
Given a QUESTION and GENERATED_RESPONSE and EXPECTED_RESPONSE.

Compare the factual content of the GENERATED_RESPONSE with the EXPECTED_RESPONSE. Ignore any differences in style, grammar, or punctuation.
  The GENERATED_RESPONSE may either be a subset or superset of the EXPECTED_RESPONSE, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
  (A) The GENERATED_RESPONSE is a subset of the EXPECTED_RESPONSE and is fully consistent with it.
  (B) The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it.
  (C) The GENERATED_RESPONSE contains all the same details as the EXPECTED_RESPONSE.
  (D) There is a disagreement between the GENERATED_RESPONSE and the EXPECTED_RESPONSE.
  (E) The answers differ, but these differences don't matter from the perspective of factuality.

Give your answer in the format "Answer: One of ABCDE, Explanation: ".

Your actual task:

QUESTION: {input_query}
GENERATED_RESPONSE: {generated_answer}
EXPECTED_RESPONSE: {expected_answer}
"""

input_query = (
    "What are the top 5 topics that were explained? Only list succinct bullet points."
)
generated_answer = """
Here are the top 5 topics that were explained in the documentation for Torchtune:

* What is LoRA and how does it work?
* Fine-tuning with LoRA: memory savings and parameter-efficient finetuning
* Running a LoRA finetune with Torchtune: overview and recipe
* Experimenting with different LoRA configurations: rank, alpha, and attention modules
* LoRA finetuning
"""
expected_answer = """LoRA"""

rows = [
    {
        "input_query": input_query,
        "generated_answer": generated_answer,
        "expected_answer": expected_answer,
    },
]

scoring_params = {
    "llm-as-judge::base": {
        "judge_model": judge_model_id,
        "prompt_template": JUDGE_PROMPT,
        "type": "llm_as_judge",
        "judge_score_regexes": ["Answer: (A|B|C|D|E)"],
    },
    "basic::subset_of": None,
}

response = client.scoring.score(input_rows=rows, scoring_functions=scoring_params)
pprint(response)


ScoringScoreResponse(
│   results={
│   │   'llm-as-judge::base': ScoringResult(
│   │   │   aggregated_results={},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 'B',
│   │   │   │   │   'judge_feedback': 'Answer: B, Explanation: The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it. The EXPECTED_RESPONSE only mentions "LoRA", which is a topic that is extensively covered in the GENERATED_RESPONSE. The GENERATED_RESPONSE provides more specific and detailed topics related to LoRA, but it does not contradict the EXPECTED_RESPONSE.'
│   │   │   │   }
│   │   │   ]
│   │   ),
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 1.0, 'num_correct': 1.0, 'num_total': 1}},
│   │   │   score_rows=[{'score': 1.0}]
│   │   )
│   }
)

## 4. Image Understanding with Llama 3.2

Below is a complete example of to ask Llama 3.2 questions about an image.

### 4.1 Setup and helpers


### 4.2 Using Llama Stack Inference API for multimodal inference

In [ ]:
vision_model_id = "groq/meta-llama/llama-4-maverick-17b-128e-instruct"
response = client.chat.completions.create(
    model=vision_model_id,
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "What's in this image?"},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://raw.githubusercontent.com/meta-llama/llama-models/refs/heads/main/Llama_Repo.jpeg",
                },
            },
        ],
    }],
)

print(response.choices[0].message.content)

The image depicts three llamas standing at a table, with one wearing a party hat and another having a purple hue. The scene is set in a barn-like environment.

*   Three llamas are positioned at a table.
    *   The llama on the left is white.
    *   The middle llama is purple.
    *   The llama on the right is white and wears a blue party hat.
*   A glass containing an orange liquid sits on the table.
    *   The glass is clear and filled with a yellowish-orange substance.
*   The background features wooden walls.
    *   The walls are composed of vertical wooden planks.
    *   The overall atmosphere suggests a celebratory or festive setting.

In summary, the image showcases three llamas gathered around a table, with one donning a party hat, amidst a rustic barn-like backdrop.
